In [20]:
import numpy as np
import random
from collections import deque

# puzzle

In [21]:
# Define the goal state for the 8-puzzle
goal_state = np.array([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 0]])  # '0' represents the blank space

# Helper function to generate a random solvable 8-puzzle
def is_solvable(puzzle):
    """ Check if the puzzle is solvable. """
    flat_puzzle = puzzle.flatten()
    inv_count = 0
    for i in range(8):
        for j in range(i + 1, 9):
            if flat_puzzle[j] and flat_puzzle[i] and flat_puzzle[i] > flat_puzzle[j]:
                inv_count += 1
    return inv_count % 2 == 0

def generate_puzzle():
    """ Generate a solvable random 8-puzzle. """
    puzzle = goal_state.copy()
    while True:
        np.random.shuffle(puzzle.flat)
        if is_solvable(puzzle) and not np.array_equal(puzzle, goal_state):
            break
    return puzzle

In [22]:
# Example puzzle generation
initial_state = generate_puzzle()
print("Initial State:")
print(initial_state)

Initial State:
[[7 1 3]
 [6 8 5]
 [4 0 2]]


C:\Users\amita\AppData\Local\Temp\ipykernel_17300\230494868.py:21: UserWarning: you are shuffling a 'flatiter' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(puzzle.flat)


# RTA*

In [23]:
# Define possible moves: up, down, left, right
moves = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}


In [24]:
def get_blank_position(state):
    """ Find the position of the blank space (0) in the puzzle. """
    return np.argwhere(state == 0)[0]

def is_valid_move(blank_pos, move):
    """ Check if the move is valid based on the blank position. """
    row, col = blank_pos
    row_move, col_move = moves[move]
    new_row, new_col = row + row_move, col + col_move
    return 0 <= new_row < 3 and 0 <= new_col < 3

def make_move(state, move):
    """ Make a move and return the new puzzle configuration. """
    blank_pos = get_blank_position(state)
    row, col = blank_pos
    row_move, col_move = moves[move]
    new_row, new_col = row + row_move, col + col_move
    
    # Swap the blank space with the target
    new_state = state.copy()
    new_state[row, col], new_state[new_row, new_col] = new_state[new_row, new_col], new_state[row, col]
    
    return new_state

def get_neighbors(state):
    """ Get all valid neighbors of the current state. """
    blank_pos = get_blank_position(state)
    neighbors = []
    
    for move in moves:
        if is_valid_move(blank_pos, move):
            new_state = make_move(state, move)
            neighbors.append((move, new_state))
    
    return neighbors

# Real-Time A* Search (RTA*)
def rta_star(initial_state, heuristic, goal_state, max_depth=300):
    state = initial_state.copy()
    path = [state]
    g_cost = 0  # Initialize cost so far
    
    while not np.array_equal(state, goal_state) and g_cost < max_depth:
        neighbors = get_neighbors(state)
        
        # Evaluate neighbors using f(n) = g(n) + h(n)
        f_values = []
        for move, neighbor in neighbors:
            g = g_cost + 1  # Increment cost
            h = heuristic(neighbor, goal_state)
            f = g + h
            f_values.append((f, neighbor, move))
        
        # Choose the neighbor with the lowest f-value
        best_f, best_neighbor, best_move = min(f_values, key=lambda x: x[0])
        
        # Update the state to the best neighbor and track the path
        state = best_neighbor
        path.append(state)
        g_cost += 1
        
    return path, g_cost

# Example heuristic: Manhattan distance
def manhattan_distance(state, goal_state):
    """ Calculate the Manhattan distance for the 8-puzzle. """
    distance = 0
    for i in range(1, 9):
        pos = np.argwhere(state == i)
        goal_pos = np.argwhere(goal_state == i)
        distance += abs(pos[0][0] - goal_pos[0][0]) + abs(pos[0][1] - goal_pos[0][1])
    return distance

In [25]:
# Test the RTA* algorithm with Manhattan heuristic
path, cost = rta_star(initial_state, manhattan_distance, goal_state)
print(f"Solution found with {cost} moves.")

Solution found with 300 moves.


In [26]:
initial_state

array([[7, 1, 3],
       [6, 8, 5],
       [4, 0, 2]])

In [27]:
get_neighbors(initial_state)


[('up',
  array([[7, 1, 3],
         [6, 0, 5],
         [4, 8, 2]])),
 ('left',
  array([[7, 1, 3],
         [6, 8, 5],
         [0, 4, 2]])),
 ('right',
  array([[7, 1, 3],
         [6, 8, 5],
         [4, 2, 0]]))]